In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from IPython.display import display, Markdown

import sys
sys.path.append('../')

from evalutils.roc import get_bootstrapped_roc_ci_curves
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from utilities import data, roc

## directory where results are
EXPERIMENT_DIR = f"/data/bodyct/experiments/lung-malignancy-fairness-shaurya"
NLST_PREDS = f"{EXPERIMENT_DIR}/nlst"

TEAMS_DIR = "C:/Users/shaur/OneDrive - Radboudumc/Documents - Master - Shaurya Gaur/General/Malignancy-Estimation Results"
NLST_PREDS = f"{TEAMS_DIR}/nlst" ## Comment out if not using Teams backup (aka Chansey is up :)
RESULTS_DIR = f"{TEAMS_DIR}/fairness-analysis-results"

## DLCST

In [2]:
dlcst_preds = pd.read_csv(f"{TEAMS_DIR}/dlcst/dlcst_thijmen_kiran_sybil_malignancy_estimation_results.csv", header=0)
dlcst_preds

,PatientID,StudyDate,SeriesInstanceUID,Age,Sex,FamilyHistoryLungCa,Emphysema,NoduleCountPerScan,sybil_year1,sybil_year2,sybil_year3,sybil_year4,sybil_year5,sybil_year6,PanCan2b,Ensemble_Kiran,thijmen_mean,label
0,4,20050124,1.2.840.113704.1.111.4964.1106577805.10,55,2,0,0,9,0.021629,0.038573,0.071919,0.079270,0.095846,0.135681,0.053366,0.082652,0.166209,0
1,35,20051208,1.2.840.113704.1.111.5776.1134059140.11,56,1,1,1,2,0.001170,0.002554,0.007835,0.011039,0.018442,0.030460,0.009543,0.000408,0.003368,0
2,38,20060109,1.2.840.113704.1.111.2004.1136823831.14,62,2,0,1,4,0.001784,0.003870,0.007835,0.012797,0.019229,0.032957,0.006734,0.002702,0.065888,0
3,47,20051214,1.2.840.113704.1.111.8148.1134579622.14,57,1,0,1,1,0.003951,0.015674,0.025373,0.034010,0.040605,0.058852,0.007944,0.084158,0.423341,0
4,56,20051213,1.2.840.113704.1.111.2744.1134487263.11,64,1,0,1,3,0.000000,0.001574,0.003791,0.006847,0.010381,0.017287,0.000899,0.000013,0.005590,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,4057,20060314,1.2.840.113704.1.111.4796.1142355218.14,69,1,0,1,2,0.168810,0.273494,0.257961,0.294720,0.327062,0.383196,0.380198,0.893933,0.682322,0
595,4063,20060313,1.2.840.113704.1.111.5104.1142267340.10,55,1,0,1,4,0.001965,0.006793,0.013618,0.017289,0.021685,0.035951,0.000000,0.000099,0.103020,0
596,4079,20060328,1.2.840.113704.1.111.1308.1143556124.11,52,2,0,0,1,0.003951,0.011459,0.025373,0.034010,0.040101,0.058852,0.039054,0.070744,0.121373,0
597,4098,20060403,1.2.840.113704.1.111.5848.1144079789.11,54,1,0,1,3,0.117795,0.167839,0.189976,0.217799,0.229753,0.300137,0.134158,0.452513,0.347016,0


In [3]:
dlcst_democols = {
    'cat': {'demo': ['Sex'], 'other': ['FamilyHistoryLungCa', 'Emphysema']},
    'num': {'demo': ['Age'], 'other': ['NoduleCountPerScan']}
}

In [4]:
dlcst_preds, dlcst_democols = data.bin_numerical_columns(dlcst_preds, dlcst_democols)
dlcst_democols

{'cat': {'demo': ['Age > 61', 'Sex'],
  'other': ['Emphysema', 'FamilyHistoryLungCa', 'NoduleCountPerScan > 1']},
 'num': {'demo': ['Age'], 'other': ['NoduleCountPerScan']}}

In [5]:
DLCST_MODELCOLS = {
    "Venkadesh": "Ensemble_Kiran",
    "de Haas": "thijmen_mean",
    "PanCan2b": "PanCan2b",
    "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

In [6]:
result_df = roc.all_results_subgroups_models(dlcst_preds, dlcst_democols['cat'], models=DLCST_MODELCOLS, csvpath=f"{RESULTS_DIR}/auroc-dlcst-{len(dlcst_preds)}.csv")
result_df

,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.909707,0.113408,True,0.923066,0.868841,0.969003,False,0.928394,0.877238,0.970634,...,133,27.378965,18.902439,28,407,72.621035,6.436782,Ensemble_Kiran,Age > 61,demo
de Haas,0.68605,0.404221,True,0.912013,0.860069,0.964839,False,0.931365,0.887097,0.97577,...,133,27.378965,18.902439,28,407,72.621035,6.436782,thijmen_mean,Age > 61,demo
PanCan2b,0.419335,0.807575,True,0.851815,0.792921,0.915105,False,0.898209,0.841399,0.945332,...,133,27.378965,18.902439,28,407,72.621035,6.436782,PanCan2b,Age > 61,demo
Sybil year 1,0.928492,0.089743,True,0.860758,0.78427,0.930278,False,0.866137,0.785758,0.923974,...,133,27.378965,18.902439,28,407,72.621035,6.436782,sybil_year1,Age > 61,demo
Venkadesh,0.437917,0.775716,1.0,0.908254,0.837521,0.970516,2.0,0.944307,0.908594,0.968446,...,291,53.923205,9.907121,27,249,46.076795,9.782609,Ensemble_Kiran,Sex,demo
de Haas,0.353367,0.928078,1.0,0.90296,0.839925,0.965447,2.0,0.946357,0.913565,0.974769,...,291,53.923205,9.907121,27,249,46.076795,9.782609,thijmen_mean,Sex,demo
PanCan2b,0.35083,-0.932981,1.0,0.909613,0.85165,0.953015,2.0,0.856363,0.792547,0.90008,...,291,53.923205,9.907121,27,249,46.076795,9.782609,PanCan2b,Sex,demo
Sybil year 1,0.88113,-0.149537,1.0,0.866212,0.77639,0.946607,2.0,0.857134,0.782818,0.922584,...,291,53.923205,9.907121,27,249,46.076795,9.782609,sybil_year1,Sex,demo
Venkadesh,0.210925,1.251025,1.0,0.91071,0.853562,0.954101,0.0,0.965888,0.940037,0.988043,...,361,67.612688,10.864198,15,179,32.387312,7.731959,Ensemble_Kiran,Emphysema,other
de Haas,0.508438,0.661272,1.0,0.915596,0.866233,0.960734,0.0,0.948117,0.902625,0.978142,...,361,67.612688,10.864198,15,179,32.387312,7.731959,thijmen_mean,Emphysema,other


## NLST

In [2]:
nlst_preds_nodule = pd.read_csv(f"{NLST_PREDS}/nlst_demov4_allmodels_cal.csv")

with open(f'{NLST_PREDS}/nlst_demo_v4_cols.json') as json_data:
    nlst_democols_original = json.load(json_data)
    json_data.close()

nlst_democols_original['cat'].pop('lungcanc')
nlst_democols_original

{'num': {'demo': ['BMI', 'Age', 'height', 'weight'],
  'smoke': ['smokeage', 'smokeday', 'smokeyr', 'pkyr'],
  'nodule': ['CoordX', 'CoordZ', 'CoordY', 'Mean_Entropy_Kiran'],
  'other': ['NoduleCounts', 'Diameter_mm']},
 'cat': {'demo': ['Overweight',
   'educat',
   'Gender',
   'Married',
   'HighSchoolPlus',
   'NonHispanicWhite',
   'Unfinished_ed',
   'WhiteOrBlack',
   'marital',
   'ethnic',
   'race'],
  'smoke': ['smokelive', 'cigar', 'cigsmok', 'smokework', 'pipe'],
  'work': ['wrkbaki',
   'wrkfoun',
   'wrkchem',
   'wrkasbe',
   'wrkfire',
   'wrksand',
   'wrkfarm',
   'wrkcoal',
   'wrkpain',
   'wrkweld',
   'wrkflou',
   'wrkbutc',
   'wrkhard',
   'wrkcott'],
  'disease': ['diagasbe',
   'diagchas',
   'diagpneu',
   'diagstro',
   'diagemph',
   'diagbron',
   'diagsili',
   'diagsarc',
   'diaghear',
   'diagdiab',
   'diagadas',
   'diagcopd',
   'diagfibr',
   'diagtube',
   'diaghype',
   'diagchro'],
  'canchist': ['canckidn',
   'cancphar',
   'canccolo',
   'c

### Nodule-level

In [ ]:
nlst_preds, nlst_democols, MODELS = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_original, scanlevel=False, tijmen=False, sybil=False)
print(len(nlst_preds), " nodules")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv")
result_df

16077  nodules


c:\Users\shaur\Documents\bodyct-lung-malignancy-fairness\utilities\roc.py:59: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.185623,1.323639,True,0.902498,0.890859,0.91109,False,0.917669,0.906832,0.927024,...,8398,57.237047,8.737231,445,6430,42.762953,6.472727,DL_cal,Age > 61,demo
de Haas Local,0.474735,0.714797,True,0.892337,0.883819,0.902983,False,0.901062,0.889233,0.912455,...,8398,57.237047,8.737231,445,6430,42.762953,6.472727,Thijmen_local_cal,Age > 61,demo
de Haas Global (hidden nodule),0.358868,0.917524,True,0.858178,0.849051,0.868449,False,0.870622,0.857232,0.882986,...,8398,57.237047,8.737231,445,6430,42.762953,6.472727,Thijmen_global_hidden_cal,Age > 61,demo
de Haas Global (shown nodule),0.187259,-1.318732,True,0.895447,0.88708,0.905291,False,0.878542,0.86528,0.891992,...,8398,57.237047,8.737231,445,6430,42.762953,6.472727,Thijmen_global_show_cal,Age > 61,demo
PanCan2b,0.485667,0.697217,True,0.83037,0.817448,0.841415,False,0.840516,0.823942,0.860333,...,8398,57.237047,8.737231,445,6430,42.762953,6.472727,PanCan2b,Age > 61,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venkadesh,0.199439,1.283152,LightSpeed QX/i,0.895073,0.869483,0.915284,Volume Zoom,0.919862,0.89897,0.937618,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,DL_cal,ManufacturersModelName,scanner
de Haas Local,0.458104,0.741973,LightSpeed QX/i,0.87524,0.851591,0.900752,Volume Zoom,0.891072,0.865797,0.913374,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.852734,0.185631,LightSpeed QX/i,0.863618,0.844484,0.883522,Volume Zoom,0.86781,0.848926,0.882675,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.744952,0.325303,LightSpeed QX/i,0.896691,0.873727,0.913598,Volume Zoom,0.903232,0.884644,0.922774,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,Thijmen_global_show_cal,ManufacturersModelName,scanner


In [ ]:
nlst_preds, nlst_democols, MODELS = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_original, scanlevel=False, tijmen=True, sybil=False)
print(len(nlst_preds), " nodules")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv")
result_df

c:\Users\shaur\Documents\bodyct-lung-malignancy-fairness\utilities\data.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[query_string] = df.eval(query_string)
c:\Users\shaur\Documents\bodyct-lung-malignancy-fairness\utilities\data.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[query_string] = df.eval(query_string)
c:\Users\shaur\Documents\bodyct-lung-malignancy-fairness\utilities\data.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

3240  nodules


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.707049,0.375823,True,0.893352,0.870166,0.916681,False,0.903191,0.876564,0.930569,...,1755,60.061728,9.815005,90,1204,39.938272,6.955178,DL_cal,Age > 61,demo
de Haas Combined,0.430028,-0.789144,True,0.912424,0.891279,0.928096,False,0.891621,0.858668,0.917424,...,1755,60.061728,9.815005,90,1204,39.938272,6.955178,Thijmen_mean_cal,Age > 61,demo
de Haas Local,0.864296,-0.170908,True,0.902554,0.871822,0.921766,False,0.898068,0.867164,0.925,...,1755,60.061728,9.815005,90,1204,39.938272,6.955178,Thijmen_local_cal,Age > 61,demo
de Haas Global (hidden nodule),0.219168,-1.228745,True,0.868102,0.847911,0.887411,False,0.829904,0.79735,0.860646,...,1755,60.061728,9.815005,90,1204,39.938272,6.955178,Thijmen_global_hidden_cal,Age > 61,demo
de Haas Global (shown nodule),0.106383,-1.614666,True,0.899443,0.877871,0.921754,False,0.852356,0.820349,0.884129,...,1755,60.061728,9.815005,90,1204,39.938272,6.955178,Thijmen_global_show_cal,Age > 61,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
de Haas Combined,0.560479,-0.58213,LightSpeed QX/i,0.916476,0.884431,0.947395,Volume Zoom,0.892479,0.832905,0.933642,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_mean_cal,ManufacturersModelName,scanner
de Haas Local,0.237697,-1.180764,LightSpeed QX/i,0.917535,0.887045,0.947729,Volume Zoom,0.866135,0.794973,0.918767,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.668197,-0.428624,LightSpeed QX/i,0.887629,0.858198,0.919354,Volume Zoom,0.868213,0.830912,0.905542,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.595375,-0.531063,LightSpeed QX/i,0.922275,0.89482,0.944243,Volume Zoom,0.901086,0.859262,0.937322,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_global_show_cal,ManufacturersModelName,scanner


### Scan-level

In [15]:
nlst_preds, nlst_democols, MODELS = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_original, scanlevel=True, tijmen=False, sybil=True)
print(len(nlst_preds), " scans")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv")
result_df

5911  scans


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.132657,1.503707,True,0.87811,0.858441,0.893875,False,0.904852,0.891247,0.930911,...,2967,56.251057,10.766917,223,2363,43.748943,8.623357,DL_cal,Age > 61,demo
de Haas Local,0.328817,0.9765,True,0.864697,0.846223,0.884755,False,0.883209,0.857003,0.908854,...,2967,56.251057,10.766917,223,2363,43.748943,8.623357,Thijmen_local_cal,Age > 61,demo
de Haas Global (hidden nodule),0.275042,1.091525,True,0.786413,0.76672,0.808836,False,0.81069,0.786524,0.836693,...,2967,56.251057,10.766917,223,2363,43.748943,8.623357,Thijmen_global_hidden_cal,Age > 61,demo
de Haas Global (shown nodule),0.589268,-0.539898,True,0.857046,0.845183,0.876881,False,0.84607,0.818877,0.86718,...,2967,56.251057,10.766917,223,2363,43.748943,8.623357,Thijmen_global_show_cal,Age > 61,demo
Sybil year 1,0.697378,0.388863,True,0.837847,0.813851,0.857848,False,0.845886,0.814478,0.877089,...,2967,56.251057,10.766917,223,2363,43.748943,8.623357,sybil_year1,Age > 61,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
de Haas Local,0.967376,0.0409,Volume Zoom,0.885999,0.846497,0.924116,Sensation 16,0.887243,0.849386,0.917924,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.951275,-0.061105,Volume Zoom,0.805651,0.769375,0.836721,Sensation 16,0.803406,0.767096,0.844879,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.488925,-0.69202,Volume Zoom,0.872788,0.837737,0.9042,Sensation 16,0.85009,0.809006,0.882188,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,Thijmen_global_show_cal,ManufacturersModelName,scanner
Sybil year 1,0.258734,-1.129388,Volume Zoom,0.884241,0.851066,0.9096,Sensation 16,0.847677,0.803974,0.891938,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,sybil_year1,ManufacturersModelName,scanner


In [3]:
nlst_preds, nlst_democols, MODELS = data.prep_nlst_preds(nlst_preds_nodule, nlst_democols_original, scanlevel=True, tijmen=True, sybil=True)
print(len(nlst_preds), " scans")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv", plot=False)
result_df

1172  scans


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.215231,1.239309,True,0.835031,0.785714,0.874194,False,0.886407,0.840167,0.933685,...,591,57.081911,11.659193,49,454,42.918089,9.741551,DL_cal,Age > 61,demo
de Haas Combined,0.727101,0.348985,True,0.862578,0.832523,0.89897,False,0.876914,0.830161,0.916705,...,591,57.081911,11.659193,49,454,42.918089,9.741551,Thijmen_mean_cal,Age > 61,demo
de Haas Local,0.458206,0.741804,True,0.855036,0.813509,0.901065,False,0.885268,0.827847,0.933729,...,591,57.081911,11.659193,49,454,42.918089,9.741551,Thijmen_local_cal,Age > 61,demo
de Haas Global (hidden nodule),0.849725,-0.189469,True,0.773673,0.734545,0.811233,False,0.764294,0.711086,0.819779,...,591,57.081911,11.659193,49,454,42.918089,9.741551,Thijmen_global_hidden_cal,Age > 61,demo
de Haas Global (shown nodule),0.849385,0.189903,True,0.814697,0.779275,0.851349,False,0.823457,0.769289,0.878068,...,591,57.081911,11.659193,49,454,42.918089,9.741551,Thijmen_global_show_cal,Age > 61,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
de Haas Local,0.565975,0.573989,Volume Zoom,0.832588,0.767969,0.888418,Sensation 16,0.871221,0.790756,0.946479,...,233,22.696246,12.406015,19,138,13.395904,12.101911,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.685043,-0.405592,Volume Zoom,0.814943,0.759566,0.867754,Sensation 16,0.784298,0.716792,0.86211,...,233,22.696246,12.406015,19,138,13.395904,12.101911,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.185487,-1.324048,Volume Zoom,0.866543,0.814904,0.918615,Sensation 16,0.768761,0.655702,0.878151,...,233,22.696246,12.406015,19,138,13.395904,12.101911,Thijmen_global_show_cal,ManufacturersModelName,scanner
Sybil year 1,0.434555,-0.781422,Volume Zoom,0.873205,0.813825,0.923301,Sensation 16,0.818425,0.709909,0.908784,...,233,22.696246,12.406015,19,138,13.395904,12.101911,sybil_year1,ManufacturersModelName,scanner


## Load a result DataFrame

In [5]:
dataset_name, dataset_len = 'dlcst', 599
filename = f"{RESULTS_DIR}/auroc-{dataset_name}-{dataset_len}.csv"
dlcst_result_df = pd.read_csv(filename, index_col=0)
dlcst_result_df.sort_values(by='p')

,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Sybil year 1,0.027959,-2.197867,False,0.920746,0.863010,0.961511,True,0.781735,0.667899,0.892308,...,298,55.592654,10.510511,24,242,44.407346,9.022556,sybil_year1,NoduleCountPerScan > 1,other
de Haas,0.189745,1.311334,0.0,0.909660,0.843920,0.953267,1.0,0.966651,0.930070,0.997565,...,454,82.971619,8.651911,16,86,17.028381,15.686275,thijmen_mean,FamilyHistoryLungCa,other
Venkadesh,0.210925,1.251025,1.0,0.910710,0.853562,0.954101,0.0,0.965888,0.940037,0.988043,...,361,67.612688,10.864198,15,179,32.387312,7.731959,Ensemble_Kiran,Emphysema,other
PanCan2b,0.350830,-0.932981,1.0,0.909613,0.851650,0.953015,2.0,0.856363,0.792547,0.900080,...,291,53.923205,9.907121,27,249,46.076795,9.782609,PanCan2b,Sex,demo
de Haas,0.353367,0.928078,1.0,0.902960,0.839925,0.965447,2.0,0.946357,0.913565,0.974769,...,291,53.923205,9.907121,27,249,46.076795,9.782609,thijmen_mean,Sex,demo
Venkadesh,0.387424,0.864298,0.0,0.914361,0.851641,0.952758,1.0,0.954671,0.917565,0.986973,...,454,82.971619,8.651911,16,86,17.028381,15.686275,Ensemble_Kiran,FamilyHistoryLungCa,other
PanCan2b,0.419335,0.807575,True,0.851815,0.792921,0.915105,False,0.898209,0.841399,0.945332,...,133,27.378965,18.902439,28,407,72.621035,6.436782,PanCan2b,Age > 61,demo
Venkadesh,0.437917,0.775716,1.0,0.908254,0.837521,0.970516,2.0,0.944307,0.908594,0.968446,...,291,53.923205,9.907121,27,249,46.076795,9.782609,Ensemble_Kiran,Sex,demo
PanCan2b,0.464761,0.731030,1.0,0.871445,0.810205,0.923183,0.0,0.915350,0.863587,0.965217,...,361,67.612688,10.864198,15,179,32.387312,7.731959,PanCan2b,Emphysema,other
de Haas,0.508438,0.661272,1.0,0.915596,0.866233,0.960734,0.0,0.948117,0.902625,0.978142,...,361,67.612688,10.864198,15,179,32.387312,7.731959,thijmen_mean,Emphysema,other


### Tijmen Combined Model

No demographic biases.

In [10]:
df = pd.read_csv(f"{RESULTS_DIR}/auroc-nlst-1172.csv", index_col=0)
df[(df['col'] == 'Thijmen_mean_cal') & (df['category'].isin(['demo', 'nodule', 'other']))].sort_values(by='p')

,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
de Haas Combined,0.039857,-2.055233,True,0.897763,0.868683,0.924716,False,0.789878,0.684623,0.875921,...,451,46.672355,17.550274,31,594,53.327645,4.960000,Thijmen_mean_cal,NoduleCounts > 1,other
de Haas Combined,0.043854,2.015484,True,0.828637,0.779566,0.875967,False,0.908389,0.876727,0.941611,...,530,51.194539,11.666667,57,515,48.805461,9.965035,Thijmen_mean_cal,NoduleInUpperLung,nodule
de Haas Combined,0.100501,-1.642430,True,0.888451,0.863206,0.919204,False,0.796294,0.692737,0.891098,...,695,67.918089,12.688442,26,350,32.081911,6.914894,Thijmen_mean_cal,Solid,nodule
de Haas Combined,0.293726,-1.049982,False,0.885508,0.844361,0.919815,True,0.842372,0.788014,0.888353,...,663,62.542662,9.549795,57,382,37.457338,12.984055,Thijmen_mean_cal,Emphysema,other
de Haas Combined,0.413132,0.818395,False,0.858478,0.816388,0.895966,True,0.894204,0.830219,0.946273,...,767,73.464164,10.917538,33,278,26.535836,10.610932,Thijmen_mean_cal,Unfinished_ed,demo
de Haas Combined,0.444389,0.764802,False,0.857575,0.811361,0.891285,True,0.889609,0.827769,0.946115,...,783,74.232082,10.000000,40,262,25.767918,13.245033,Thijmen_mean_cal,FamilyHistoryLungCa,other
de Haas Combined,0.615203,0.502660,3.0,0.864932,0.811988,0.916232,5.0,0.891964,0.836793,0.940720,...,271,26.194539,11.726384,31,213,20.819113,12.704918,Thijmen_mean_cal,educat,demo
de Haas Combined,0.628688,-0.483574,1.0,0.876015,0.845255,0.906084,2.0,0.835237,0.714844,0.941176,...,986,93.686007,10.200364,9,31,3.412969,22.500000,Thijmen_mean_cal,race,demo
de Haas Combined,0.628688,-0.483574,1.0,0.876015,0.845255,0.906084,2.0,0.835237,0.714844,0.941176,...,986,93.686007,10.200364,9,31,3.412969,22.500000,Thijmen_mean_cal,WhiteOrBlack,demo
de Haas Combined,0.638692,-0.469528,True,0.872882,0.835344,0.900496,False,0.842990,0.729798,0.926809,...,978,92.918089,10.192837,16,67,7.081911,19.277108,Thijmen_mean_cal,NonHispanicWhite,demo


In [11]:
df = pd.read_csv(f"{RESULTS_DIR}/auroc-nlst-3240.csv", index_col=0)
df[(df['col'] == 'Thijmen_mean_cal') & (df['category'].isin(['demo', 'nodule', 'other']))].sort_values(by='p')

,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
de Haas Combined,0.003215,-2.946373,True,0.934576,0.920796,0.948376,False,0.842466,0.793360,0.885562,...,1920,65.740741,9.859155,71,1039,34.259259,6.396396,Thijmen_mean_cal,NoduleCounts > 1,other
de Haas Combined,0.003537,-2.916757,True,0.930298,0.916818,0.946092,False,0.837235,0.793015,0.883949,...,1773,61.234568,10.635081,70,1186,38.765432,5.573248,Thijmen_mean_cal,Solid,nodule
de Haas Combined,0.015366,2.423644,True,0.870728,0.846650,0.891844,False,0.929098,0.907180,0.950862,...,1206,42.407407,12.227074,113,1753,57.592593,6.055734,Thijmen_mean_cal,NoduleInUpperLung,nodule
de Haas Combined,0.079190,1.755405,True,0.848969,0.821546,0.876165,False,0.907649,0.872761,0.940896,...,1089,40.895062,17.811321,45,1870,59.104938,2.349869,Thijmen_mean_cal,Diameter_mm > 7,other
de Haas Combined,0.131407,-1.508576,1.0,0.919542,0.904907,0.934459,2.0,0.880503,0.850714,0.914586,...,1665,56.913580,9.707158,102,1294,43.086420,7.306590,Thijmen_mean_cal,Gender,demo
de Haas Combined,0.132421,-1.504625,False,0.919936,0.900614,0.937372,True,0.883060,0.851958,0.908597,...,1962,65.370370,7.365439,125,997,34.629630,11.140820,Thijmen_mean_cal,Emphysema,other
de Haas Combined,0.275438,-1.090625,True,0.908977,0.887443,0.925551,False,0.863247,0.807692,0.914262,...,2746,92.407407,8.283233,33,213,7.592593,13.414634,Thijmen_mean_cal,NonHispanicWhite,demo
de Haas Combined,0.340366,0.953443,False,0.898140,0.873895,0.920726,True,0.922188,0.893418,0.945511,...,2141,72.283951,8.582408,80,818,27.716049,8.908686,Thijmen_mean_cal,Unfinished_ed,demo
de Haas Combined,0.370772,-0.895027,True,0.913618,0.896356,0.930921,False,0.890228,0.862594,0.920324,...,1994,67.345679,8.615949,93,965,32.654321,8.790170,Thijmen_mean_cal,Married,demo
de Haas Combined,0.377535,-0.882448,False,0.909540,0.891144,0.924507,True,0.862741,0.776389,0.927098,...,2825,95.246914,8.457550,20,134,4.753086,12.987013,Thijmen_mean_cal,PersonalCancerHist,other


### Other models - scan level.

In [18]:
filename = f"{RESULTS_DIR}/auroc-nlst-5911.csv"
df = pd.read_csv(filename, index_col=0)
df[df['category'].isin(['demo'])].sort_values(by='p')[['attribute', 'p', 
                                                       'Group_2', 'Group_2_mal', 'AUC_2', 'AUC-CI-lo_2', 'AUC-CI-hi_2', 
                                                       'Group_1', 'Group_1_mal', 'AUC_1', 'AUC-CI-lo_1', 'AUC-CI-hi_1', 
                                                       ]]

,attribute,p,Group_2,Group_2_mal,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,Group_1,Group_1_mal,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1
Sybil year 1,height > 68,0.000323,True,267,0.800426,0.773187,0.824131,False,314,0.873697,0.851941,0.893239
Sybil year 1,Gender,0.000548,2.0,244,0.880180,0.858218,0.902326,1.0,337,0.811671,0.788581,0.837966
Sybil year 1,marital,0.000649,5.0,125,0.900503,0.874532,0.929432,2.0,384,0.824532,0.799279,0.846373
PanCan2b,Overweight,0.000797,False,209,0.729121,0.695184,0.763285,True,372,0.808030,0.788701,0.830220
Sybil year 1,ethnic,0.004487,1.0,4,0.981050,0.956522,1.000000,2.0,574,0.841064,0.823057,0.857982
...,...,...,...,...,...,...,...,...,...,...,...,...
de Haas Local,Overweight,0.964593,False,209,0.870638,0.849109,0.890377,True,372,0.869771,0.851916,0.887000
de Haas Global (shown nodule),Unfinished_ed,0.977862,True,158,0.852619,0.824059,0.883263,False,423,0.852006,0.837972,0.870241
Sybil year 1,NonHispanicWhite,0.979297,False,55,0.843590,0.783983,0.902353,True,526,0.844478,0.823266,0.863796
Venkadesh,Gender,0.988677,2.0,244,0.889530,0.871460,0.910717,1.0,337,0.889276,0.873224,0.905323


### Other models - nodule level.

In [19]:
filename = f"{RESULTS_DIR}/auroc-nlst-16077.csv"
df = pd.read_csv(filename, index_col=0)
df[df['category'].isin(['demo'])].sort_values(by='p')[['attribute', 'p', 
                                                       'Group_2', 'Group_2_mal', 'AUC_2', 'AUC-CI-lo_2', 'AUC-CI-hi_2', 
                                                       'Group_1', 'Group_1_mal', 'AUC_1', 'AUC-CI-lo_1', 'AUC-CI-hi_1', 
                                                       ]]

,attribute,p,Group_2,Group_2_mal,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,Group_1,Group_1_mal,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1
de Haas Global (shown nodule),weight > 180,0.006623,True,502,0.907268,0.894433,0.917750,False,747,0.874493,0.863796,0.887540
de Haas Local,ethnic,0.020337,1.0,9,0.977523,0.954774,1.000000,2.0,1230,0.895624,0.884470,0.905556
de Haas Global (shown nodule),Married,0.024447,False,453,0.870069,0.857133,0.882460,True,796,0.899105,0.888840,0.907624
PanCan2b,ethnic,0.029651,1.0,9,0.948307,0.901603,0.983000,2.0,1230,0.835222,0.824549,0.845057
de Haas Global (shown nodule),Overweight,0.049413,False,445,0.873113,0.858090,0.888516,True,804,0.898451,0.888532,0.905758
...,...,...,...,...,...,...,...,...,...,...,...,...
de Haas Global (shown nodule),Unfinished_ed,0.938549,True,378,0.890066,0.875193,0.906598,False,871,0.889055,0.878893,0.897789
de Haas Global (hidden nodule),educat,0.943673,5.0,304,0.861452,0.845872,0.878091,3.0,311,0.860126,0.844006,0.872611
de Haas Global (shown nodule),HighSchoolPlus,0.966633,False,24,0.888567,0.840524,0.938017,True,1225,0.890413,0.882709,0.898975
de Haas Local,Married,0.982773,False,453,0.895847,0.883826,0.909235,True,796,0.896111,0.886541,0.905217
